In [ ]:
# En esta celda se puede probar la funcion de analisis de una imagen

from lib.image import image_data
from lib.analysis import dyssynchonia_analysis
import cv2

# Path completo donde esta la imagen (incluyendo nombre y extencion)
path = 'photos_examples/c1d000.tif'

# Ancho de la feta a analisar (es un parametro opcional)
ancho_corte = 5

# Distancia minima en pixeles entre picos
min_dist_between_maxs = 20

image = cv2.imread(path)
# Select ROI
fromCenter = False
showCrosshair = False
seleted_parameters = cv2.selectROI(image, fromCenter, showCrosshair)

# Crop image
x_start = int(seleted_parameters[1])
x_end = x_start + int(seleted_parameters[3])
y_start = int(seleted_parameters[0])
y_end = y_start + int(seleted_parameters[2])

cv2.destroyAllWindows()

image = image_data.get_image_data(image)
image = image_data.crop_vertical(image, x_start, x_end)
image = image_data.crop_horizontal(image, y_start, y_end)

results = dyssynchonia_analysis.analyze_image(image, min_dist_between_maxs, ancho_corte)

# PRUEBA ANALISIS 

In [7]:
# En esta celda se puede probar la funcion de analisis todas las imagenes .tif en un directorio
import os
from lib.image import image_data
from lib.analysis import dyssynchonia_analysis
import cv2

# Ancho de la feta a analisar (es un parametro opcional)
ancho_corte = 5

# Distancia minima en pixeles entre picos
min_dist_between_maxs = 20


# Path donde estan las imagenes
path = "C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/seleccion/050320sel/050320c1"

images_paths = [ f'{path}/{file}' for file in os.listdir(path) if file.endswith(".tif") ]
images = [ cv2.imread(image_path) for image_path in images_paths ]

In [8]:
fromCenter = False
showCrosshair = False

#Tomamos la primer imagen para seleccionar el recorte elegido el resto de las imagenes se van a recortar igual
seleted_parameters = cv2.selectROI(images[0], fromCenter, showCrosshair)

# Crop image
x_start = int(seleted_parameters[1])
x_end = x_start + int(seleted_parameters[3])
y_start = int(seleted_parameters[0])
y_end = y_start + int(seleted_parameters[2])

cv2.destroyAllWindows()

In [9]:
images = [ image_data.get_image_data(image) for image in images ]
images = [ image_data.crop_vertical(image, x_start, x_end) for image in images ]
images = [ image_data.crop_horizontal(image, y_start, y_end) for image in images ]

In [10]:
dy_analysis = [dyssynchonia_analysis.analyze_image(image, min_dist_between_maxs, ancho_corte) for image in images]

In [11]:
with open(path + r'test.txt', 'w') as f:
    f.write(" ".join(map(str, dy_analysis)))

# Analisis de datos
Notas: 
- Image corresponde al análisis de toda la célula y slices corresponde a fetas de esa imagen
- intensidades es el valor que resulta de comprimir (sumando) la matriz con las coordenadas de x,y,z de los pixeles 
- max_peaks_pos es la index de en la lista de intensidades 
- max_peaks_intensities el valor de intensidad que se corresponde con el pico en la posicion analoga de max_peaks_pos

In [12]:
#Imports
import json
import numpy as np
import pandas as pd

In [13]:
#encodes a dictionary into a jason
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


def jasonGenerator(path,results):
    """ Generates a .jason file in the path given folder from the a dictionary """
    with open(path + '/analysis_result.jason',"w") as miarch:
        miarch.write(json.dumps(results, cls=NumpyEncoder))

In [14]:
def jasonReader(path):
    """ Reads the .jason generated with jasonGenartor the given path"""
    with open( path + '/analysis_result.json','r') as miarch:
        loaded_dict = json.loads(miarch.read())
    return loaded_dict

In [15]:
def slicesParser_max_peaksI(path):
    #read the analysis result 
    dictres = jasonReader(path)

    #extract all the slices data
    allSlices = dictres['slices']
    
    try:
        #seting columns names
        column_names = ['transient' + str(x) for x in range(1,len(allSlices[0]['max_peaks_intensities']))]

        #define a dataframe
        df_sum = pd.DataFrame(columns=column_names)

        for i in range(0,len(allSlices)):
            df_sum.loc[i+1] = allSlices[i]['max_peaks_intensities'][1:]

        return df_sum
    
    except:
        print('no possible callculation')

In [16]:
def slicesParser_halfpeakstime(path):
    #read the analysis result 
    dictres = jasonReader(path)
    #extract all the slices data
    allSlices = dictres['slices']
    try:
        #seting columns names
        column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['times_to_half_peaks']))]
        #define a dataframe
        df_sum = pd.DataFrame(columns=column_names)
        #populating the dataframe
        for i in range(0,len(allSlices)):
            df_sum.loc[i+1] = allSlices[i]['times_to_half_peaks']

        return df_sum

    except:
        print('no possible callculation')

In [17]:
def slicesParser_amplitudes(path):
    #read the analysis result 
    dictres = jasonReader(path)
    #extract all the slices data
    allSlices = dictres['slices']
    try:
        #seting columns names
        column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['amplitudes']))]
        #define a dataframe
        df_sum = pd.DataFrame(columns=column_names)
        #populating the dataframe
        for i in range(0,len(allSlices)):
            df_sum.loc[i+1] = allSlices[i]['amplitudes']

        return df_sum
    
    except:
        print('no possible callculation')

In [18]:
def slicesParser_min_peaksI(path):
    #read the analysis result 
    dictres = jasonReader(path)
    #extract all the slices data
    allSlices = dictres['slices']
    try:
        #seting columns names
        column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['min_peaks_intensities']))]
        #define a dataframe
        df_sum = pd.DataFrame(columns=column_names)
        #populating the dataframe
        for i in range(0,len(allSlices)):
            df_sum.loc[i+1] = allSlices[i]['min_peaks_intensities']

        return df_sum
    
    except:
        print('no possible callculation')

In [19]:
def slicesParser_peaktime(path):
    #read the analysis result 
    dictres = jasonReader(path)
    #extract all the slices data
    allSlices = dictres['slices']

    try:
        #seting columns names
        column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['times_to_peaks']))]
        #define a dataframe
        df_sum = pd.DataFrame(columns=column_names)
        #populating the dataframe
        for i in range(0,len(allSlices)):
            df_sum.loc[i+1] = allSlices[i]['times_to_peaks']

        return df_sum
    
    except:
        print('no possible callculation')

In [20]:
def slicesParser_tau(path):
    #read the analysis result 
    dictres = jasonReader(path)
    #extract all the slices data
    allSlices = dictres['slices']
    #seting columns names
    try:
        column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['tau_s']))]
        #define a dataframe
        df_sum = pd.DataFrame(columns=column_names)
        #populating the dataframe
        for i in range(0,len(allSlices)):
            df_sum.loc[i+1] = allSlices[i]['tau_s']
        
        return df_sum
    
    except:
        print('no possible calculation')

In [21]:
def BasicStasts(df):
    if df.empty == False:
        return df.astype('int').describe()

In [22]:
def discordIndex(df):
    return df.iloc[2].mean()

In [23]:
def amplitudes_ratio(df):
    df_alt = pd.DataFrame()
    for i in range(0,len(list(df))-1):
        name = 'ratio' + str(i+1) +'-' + str(i)
        df_alt[name] = df[list(df)[i+1]]/df[list(df)[i]]
    return df_alt

In [23]:
path_arch = path

BasicStasts(slicesParser_max_peaksI(path_arch))

,transient1,transient2,transient3,transient4,transient5
count,47.000000,47.000000,47.000000,47.000000,47.000000
mean,47.148936,47.468085,46.765957,47.042553,46.914894
std,3.413393,2.619750,3.258519,3.368379,2.857711
min,40.000000,42.000000,41.000000,39.000000,40.000000
25%,45.000000,46.000000,45.000000,45.000000,45.000000
50%,46.000000,47.000000,46.000000,47.000000,46.000000
75%,49.000000,48.000000,50.000000,49.000000,50.000000
max,54.000000,53.000000,53.000000,55.000000,53.000000


In [35]:
amplitudes_ratio(slicesParser_amplitudes(path_arch)).head(2)

,ratio1-0,ratio2-1,ratio3-2,ratio4-3
1,0.736631,1.213793,1.284722,0.802703
2,0.824930,0.967742,1.289474,0.775510


In [25]:
slicesParser_min_peaksI(path_arch).head(2)

,transient1,transient2,transient3,transient4,transient5
1,19,22,20,18,20
2,19,21,21,19,21


In [26]:
slicesParser_tau(path_arch)

no possible callculation


In [48]:
# Calculate Discordance Index

import statistics

WC_amplitud = jasonReader(path_arch)['image']['amplitudes']
local_AR = amplitudes_ratio(slicesParser_amplitudes(path_arch))
DI = local_AR.std()/statistics.mean(WC_amplitud)

In [27]:
import os
import statistics

list_DI = []
fotos_list = os.listdir(path)
fotos_list = [x.split('.')[0] for x in fotos_list if 'tif' in x]
for nombre_foto in fotos_list[:]:
    jasonGenerator(path,  'test.txt')
    WC_amplitud = jasonReader(path)['image']['amplitudes']
    local_AR = amplitudes_ratio(slicesParser_amplitudes(path))
    DI = local_AR.std()/statistics.mean(WC_amplitud)
    list_DI.append (DI)

In [28]:
list_DI

[ratio1-0    0.101236
 ratio2-1    0.104679
 ratio3-2    0.114077
 ratio4-3    0.101384
 dtype: float64,
 ratio1-0    0.101236
 ratio2-1    0.104679
 ratio3-2    0.114077
 ratio4-3    0.101384
 dtype: float64,
 ratio1-0    0.101236
 ratio2-1    0.104679
 ratio3-2    0.114077
 ratio4-3    0.101384
 dtype: float64,
 ratio1-0    0.101236
 ratio2-1    0.104679
 ratio3-2    0.114077
 ratio4-3    0.101384
 dtype: float64]

In [62]:
arch